In [1]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [2]:
pip install ktrain

In [0]:
import tensorflow as tf
import ktrain
from ktrain import text
import csv
import numpy as np

In [0]:
def extract_data(file_name):
    with open(file_name,"r") as data:
        train = csv.DictReader(data, delimiter='\t')
        sentences = []
        labels = []
        for row in train:
            sentences.append(row["sentence_A"]+". "+row["sentence_B"])
            if row["entailment_judgment"] == 'ENTAILMENT':
                labels.append(0)
            elif row["entailment_judgment"] == 'NEUTRAL':
                labels.append(1)
            elif row["entailment_judgment"] == 'CONTRADICTION':
                labels.append(2)
        return sentences, labels

In [24]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
x_train, y_train = extract_data("/content/SICK_train.txt")

x_test, y_test = extract_data("/content/SICK_trial.txt")
class_names = ['ENTAILMENT', 'NEUTRAL', 'CONTRADICTION']

In [11]:
MODEL_NAME = 'bert-base-uncased'
t = text.Transformer(MODEL_NAME, maxlen=500, class_names=class_names)
trn = t.preprocess_train(x_train, y_train)
val = t.preprocess_test(x_test, y_test)
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6)


preprocessing train...
language: en
train sequence lengths:
	mean : 19
	95percentile : 31
	99percentile : 39


preprocessing test...
language: en
test sequence lengths:
	mean : 20
	95percentile : 33
	99percentile : 43


In [16]:
learner.fit_onecycle(2e-5, 3)



begin training using onecycle policy with max lr of 2e-05...
Train for 750 steps, validate for 84 steps
Epoch 1/3
750/750 [==============================] - 757s 1s/step - loss: 0.2828 - accuracy: 0.8962 - val_loss: 0.4362 - val_accuracy: 0.8340
Epoch 2/3
750/750 [==============================] - 756s 1s/step - loss: 0.2121 - accuracy: 0.9282 - val_loss: 0.3856 - val_accuracy: 0.8680
Epoch 3/3
750/750 [==============================] - 757s 1s/step - loss: 0.1283 - accuracy: 0.9622 - val_loss: 0.4410 - val_accuracy: 0.8580


In [17]:
learner.validate(class_names=t.get_classes())

               precision    recall  f1-score   support

   ENTAILMENT       0.78      0.85      0.81       144
      NEUTRAL       0.89      0.86      0.88       282
CONTRADICTION       0.91      0.86      0.89        74

     accuracy                           0.86       500
    macro avg       0.86      0.86      0.86       500
 weighted avg       0.86      0.86      0.86       500



array([[122,  22,   0],
       [ 33, 243,   6],
       [  2,   8,  64]])

In [0]:
predictor = ktrain.get_predictor(learner.model, preproc=t)

In [0]:
predictor.save('bert_predictor_for_entailment')

In [0]:
def calculate_accuracy(predictions,labels):
    predictions = np.array(predictions)
    labels = np.array(labels)
    numbers = len(predictions)
    accuracy_sum = 0
    predictions_entailment = []
    labels_entailment = []
    predictions_neutral = []
    labels_neutral = []
    predictions_contradiction = []
    labels_contradiction = []
    for i in range(numbers):
        if(predictions[i] == labels[i]):
            accuracy_sum += 1
    return accuracy_sum

In [0]:
def model_prediction():
    # reload the predictor
    reloaded_predictor = ktrain.load_predictor('/content/bert_predictor_for_entailment')

    x_test, y_test = extract_data("/content/SICK_test_annotated.txt")

    y_test = np.array(y_test)
    prediction = reloaded_predictor.predict(x_test)
    prediction_labels = []
    for i in prediction:
        if i == 'ENTAILMENT':
            prediction_labels.append(0)
        elif i == 'NEUTRAL':
            prediction_labels.append(1)
        elif i == 'CONTRADICTION':
            prediction_labels.append(2)
    prediction_labels = np.array(prediction_labels)
    accuracy_sum = calculate_accuracy(prediction_labels,y_test)
    accuracy = accuracy_sum/len(prediction)
    print("Accuracy is:")
    print(accuracy)

In [35]:
model_prediction()